In [3]:
import pandas as pd
from random import randint
import sys
import copy

L = int(sys.argv[1])
K = int(sys.argv[2])
training_set_path = sys.argv[3]
validation_set_path = sys.argv[4]
test_set_path = sys.argv[5]
to_print_input = sys.argv[6]

training_set = pd.read_csv(training_set_path)
test_set = pd.read_csv(test_set_path)
validation_set = pd.read_csv(validation_set_path)

if to_print_input == "yes":
    to_print = True
elif to_print_input == "no":
    to_print = False
else:
    print("to-print must be 'yes' or 'no', printing anyway")
to_print = True

def node_formation(df):
    """
    recursive function which drills down until a pure node is returned
    :param df:
    :return:
    """

    # Check current variance
    variance = variance_impurity(df)

    if variance == 0:
        # Pure node
        if (df['Class'] == 0).sum() == 0:
            class_outcome = 0
        else:
            class_outcome = 1

        return {'Class': class_outcome}

    else:
        node_attribute, zero, one = gain(df)  # returns attribute to split on, and if zero or ones were present
        if zero:
            branch_0 = node_formation(df[df[node_attribute] == 0].drop([node_attribute], axis=1))
            # filters df by attribute value and drops the named attribute
        if one:
            branch_1 = node_formation(df[df[node_attribute] == 1].drop([node_attribute], axis=1))

        if zero and one:  # attribute contained both
            return {node_attribute: {0: branch_0, 1: branch_1}}
        if zero:
            return {node_attribute: {0: branch_0}}
        if one:
            return {node_attribute: {1: branch_1}}


def variance_impurity(df):
    """
    Input is dataframe, which may be subset of larger set
    VI(S) = (K0/K*K1/K)
    K0, class = 0
    K1, class = 1
    K = sum(K0 + K)
    Returns variance impurity
    """

    K0 = (df['Class'] == 0).sum()  # Turns df in True/False
    K1 = (df['Class'] == 1).sum()
    K = K0 + K1

    if K == 0:
        # print("K is 0, returning 0")
        return 0

    K2 = K0 / K
    # print("K2:")
    # print(K2)
    K3 = K1 / K
    # print("K3:")
    # print(K3)

    variance = (K2 * K3)
    # print("variance:")
    # print(variance)

    return variance


def count_01s(df):
    """
    Input is dataframe, which may be subset of larger set
    Creates a dictionary for each remaining attributes/elements in the format
    {attribute: (zeros, ones)}
    returns dictionary
    """

    attribute_counts = {}
    # dictionary for attributes with counts as tuples (0, 1)
    # This creates the elements, counts, feel free to pop out
    for attribute in df.loc[:, df.columns != ('Class' or '' or 'None')]:
        # loops over all attributes not identified as Class, the outcome
        zeros = (df[attribute] == 0).sum()  # Turns df in True/False
        ones = (df[attribute] == 1).sum()
        attribute_counts[attribute] = (zeros, ones)

    return attribute_counts


def gain(df):
    """
    Input df
    variance_impurity and count_01 in finding the maximum gain
    Returns the attribute/element which the highest gain
    """

    target_values = count_01s(df)
    VIS = variance_impurity(df)
    initialize = True

    for attribute in target_values:
        zeros = target_values[attribute][0]
        ones = target_values[attribute][1]
        Pr0 = zeros / (zeros + ones)
        Pr1 = 1 - Pr0  # Using axioms, probability will sum to 1
        VIS0 = variance_impurity(df[df[attribute] == 0])
        VIS1 = variance_impurity(df[df[attribute] == 1])
        gainSX = VIS - Pr0 * VIS0 - Pr1 * VIS1

        if initialize:
            max_gain = (attribute, gainSX)
            initialize = False
        if gainSX > max_gain[1]:
            max_gain = (attribute, gainSX)

    attribute = max_gain[0]
    zero, one = False, False
    if target_values[attribute][0] > 0:
        zero = True
    if target_values[attribute][1] > 0:
        one = True

    return attribute, zero, one

def print_decision_tree(layer_count, dictionary):
    """
    Recursively go through the dictionaries and print out the keys/values, adding space to every layer for a tiered visual
    :param layer_count: the depth of the layer, initially use -1, gets added to every recursion
    :param dictionary: the dictionary to print, might not actually be a dictionary if we're at the end of the layers
    :return: a printed decision tree
    """
    if type(dictionary) != dict:
        # print("dictionary not a dictionary, return")
        return
    layer_count = layer_count + 1

    for key, value in dictionary.items():

        # for i in range(0, layer_count): print("| ", end="")

        # print("key {} ".format(key))

        if type(value) == dict:
            for v in value:
                print("")
                for i in range(0, layer_count): print("| ", end="")
                print("{} = {} : ".format(key, v), end="")

                print_decision_tree(layer_count, value[v])
        else:
            # for i in range(0, layer_count): print("| ", end="")
            # print("value not a dictionary, end of tree?")
            print("{} ".format(value), end="")


node_dict = node_formation(training_set)

if to_print:
    print_decision_tree(-1, node_dict)

#I create two dictionaries: one for the Variance Impurity Tree and one for the IG3 Tree
n = 0
node_position = 0
node_variance = 0

labelValues = list(training_set.columns.values)
labelValues.remove('Class')

#I create two dictionaries: one for the Variance Impurity Tree and one for the IG3 Tree
def counts_in_list(seq, return_counts=False, id=None):
   
    found = set()
    if id is None:
        for x in seq:
            found.add(x)
           
    else:
        for x in seq:
            x = id(x)
            if x not in found:
                found.add(x)
    found = list(found)           
    counts = [seq.count(0),seq.count(1)]
    if return_counts:
        return found,counts
    else:
        return found

#start with variance impurity tree
def calculate_variance(target_values):
    values = list(target_values)
    elements,counts = counts_in_list(values,True)
    variance_impurity = 0
    sum_counts = sum(counts)
    for i in elements:
        variance_impurity += (-counts[i]/sum_counts*(counts[i]/sum_counts))
    return variance_impurity

def variance_impurity_gain(data, split_attribute_name, target_attribute_name):
    data_split = data.groupby(split_attribute_name)
    data_subgroup = data_split.agg({target_attribute_name : [calculate_variance, lambda x: len(x)/(len(data.index) * 1.0)] })[target_attribute_name]
    data_subgroup.columns = ['Variance', 'Observations']
    weighted_variance_impurity = sum( data_subgroup['Variance'] * data_subgroup['Observations'] )
    total_variance_impurity = calculate_variance(data[target_attribute_name])
    variance_impurity_gain = total_variance_impurity - weighted_variance_impurity
    return variance_impurity_gain

def tree_with_variance_impurity_algorithm(data, target_attribute_name, attribute_names, default_class=None):
    global node_variance
    from collections import Counter
    count_target_attributes = Counter(x for x in data[target_attribute_name])
    if len(count_target_attributes) == 1:
        return list(count_target_attributes.keys())[0]

    elif data.empty or (not attribute_names):
        return default_class 
    
    else:
        index_of_max = list(count_target_attributes.values()).index(max(count_target_attributes.values())) 
        default_class = list(count_target_attributes.keys())[index_of_max]
        variance_gain = [variance_impurity_gain(data, attr, target_attribute_name) for attr in attribute_names]
        index_of_max = variance_gain.index(max(variance_gain)) 
        best_attr = attribute_names[index_of_max]
        
        tree = {best_attr:{}}
        positiveCount = data['Class'].value_counts()[1];
        negativeCount = data['Class'].value_counts()[0];
        if positiveCount>negativeCount :
            best_class = 1
        elif positiveCount<negativeCount:
            best_class = 0
        else:
            best_class = 'none'
        tree[best_attr]['best_class'] = best_class
        node_variance = node_variance + 1
        tree[best_attr]['number'] = node_variance
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]

        for attr_val, data_subset in data.groupby(best_attr):
            subtree = tree_with_variance_impurity_algorithm(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

#now I create a dictionary for the IG3 Tree    
def calculate_entropy(probablities):
    import math
    sum_of_probablities = 0;
    for prob in probablities:
        sum_of_probablities += -prob*math.log(prob, 2)
    return sum_of_probablities

def calculate_entropy_of_the_list(list):
    from collections import Counter  
    counter = Counter(x for x in list)
    num_instances = len(list)*1.0
    probs = [x / num_instances for x in counter.values()]
    return calculate_entropy(probs)
    
def information_gain(data, split_attribute_name, target_attribute_name):
    data_split = data.groupby(split_attribute_name) 
    data_subgroup = data_split.agg({target_attribute_name : [calculate_entropy_of_the_list, lambda x: len(x)/(len(data.index) * 1.0)] })[target_attribute_name]
    data_subgroup.columns = ['Entropy', 'ProbObservations']
    new_entropy = sum( data_subgroup['Entropy'] * data_subgroup['ProbObservations'] )
    old_entropy = calculate_entropy_of_the_list(data[target_attribute_name])
    return old_entropy-new_entropy

def tree_with_IG3_algorithm(data, target_attribute_name, attribute_names, default_class=None):
    from collections import Counter
    count_target_attributes = Counter(x for x in data[target_attribute_name])
    global node_position
    if len(count_target_attributes) == 1:
        return list(count_target_attributes.keys())[0]

    elif data.empty or (not attribute_names):
        return default_class 
    
    else:
        index_of_max = list(count_target_attributes.values()).index(max(count_target_attributes.values())) 
        default_class = list(count_target_attributes.keys())[index_of_max]  
        
        info_gain = [information_gain(data, attr, target_attribute_name) for attr in attribute_names]
        index_of_max = info_gain.index(max(info_gain)) 
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}}
        positiveCount = data['Class'].value_counts()[1];
        negativeCount = data['Class'].value_counts()[0];
        if positiveCount>negativeCount :
            best_class = 1
        elif positiveCount<negativeCount:
            best_class = 0
        else:
            best_class = 'none'
        tree[best_attr]['best_class'] = best_class
        node_position = node_position + 1
        tree[best_attr]['number'] = node_position
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]

        for attr_val, data_subset in data.groupby(best_attr):
            
            subtree = tree_with_IG3_algorithm(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree
    

#now I save the dictionaries of the trees built with IG3 and Variance Impurityree = tree_with_IG3_algorithm(training_set, 'Class', labelValues)
tree_gain = tree_with_IG3_algorithm(training_set, 'Class', labelValues)
tree_variance = tree_with_variance_impurity_algorithm(training_set, 'Class', labelValues)

#The following function computes the accuracy of the tree
def tree_accuracy(instance, tree, default=None):
    attribute = list(tree.keys())[0]
    if instance[attribute] in tree[attribute].keys():
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): 
            return tree_accuracy(instance, result)
        else:
            return result 
    else:
        return default

#Now I start with the pruning algorithm

#This first function orders the nodes in the new_tree D′ from 1 to N;
def order_of_the_nodes (temp_tree, number):
    if isinstance(temp_tree, dict):
        attribute = list(temp_tree.keys())[0]
        if temp_tree[attribute]['number'] == number:
            if(temp_tree[attribute][0]!=0 and temp_tree[attribute][0]!=1):
                new_tree = temp_tree[attribute][0]
                if isinstance(new_tree, dict):
                    temp_attribute = list(new_tree.keys())[0]
                    temp_tree[attribute][0] = new_tree[temp_attribute]['best_class']
            elif(temp_tree[attribute][1]!=0 and temp_tree[attribute][1]!=1):
                new_tree = temp_tree[attribute][1]
                if isinstance(new_tree, dict):
                    temp_attribute = list(new_tree.keys())[0]      
                    temp_tree[attribute][1] = new_tree[temp_attribute]['best_class']
        else:
            left = temp_tree[attribute][0]
            right = temp_tree[attribute][1]
            order_of_the_nodes(left, number)
            order_of_the_nodes(right,number )
    return temp_tree



def number_of_internal_nodes(tree):
    if isinstance(tree, dict):
        attribute = list(tree.keys())[0]
        left = tree[attribute][0]
        right = tree[attribute][1]
        return (1 + number_of_internal_nodes(left) +  
               number_of_internal_nodes(right)); 
    else:
        return 0;
    

def post_pruning(L, K, tree):
    best_tree = tree
    for i in range(1, L+1) :
        new_tree = copy.deepcopy(best_tree)
        M = randint(1, K);
        for j in range(1, M+1):
            n = number_of_internal_nodes(new_tree)
            if n> 0:
                P = randint(1,n)
            else:
                P = 0
            order_of_the_nodes(new_tree, P)
        test_set['accuracyBeforePruning'] = test_set.apply(tree_accuracy, axis=1, args=(best_tree,'1') ) 
        accuracyBeforePruning = str( sum(test_set['Class']==test_set['accuracyBeforePruning'] ) / (1.0*len(test_set.index)) )
        test_set['accuracy_after_pruning'] = test_set.apply(tree_accuracy, axis=1, args=(new_tree,'1') ) 
        accuracy_after_pruning = str( sum(test_set['Class']==test_set['accuracy_after_pruning'] ) / (1.0*len(test_set.index)) )
        if accuracy_after_pruning >= accuracyBeforePruning:
            best_tree = new_tree
    return best_tree


#if to_print == 'yes':
#    print(tree)
#    print(tree2)
   
test_set['predicted_tree_gain'] = test_set.apply(tree_accuracy, axis=1, args=(tree_gain,'1') ) 
print( 'Accuracy with IG3 ' +  (str( sum(test_set['Class']==test_set['predicted_tree_gain'] ) / (0.01*len(test_set.index)) )))


test_set['predicted_variance_impuruty'] = test_set.apply(tree_accuracy, axis=1, args=(tree_variance,'1') ) 
print( 'Accuracy with Variance Impuruty ' + (str( sum(test_set['Class']==test_set['predicted_variance_impuruty'] ) / (0.01*len(test_set.index)) )))


pruned_tree_gain = post_pruning(L,K,tree_gain)
pruned_tree_variance = post_pruning(L,K,tree_variance)

test_set['predicted_pruned_tree_gain'] = test_set.apply(tree_accuracy, axis=1, args=(pruned_tree_gain,'1') ) 
print( 'Accuracy with pruned Information gain tree ' + (str( sum(test_set['Class']==test_set['predicted_pruned_tree_gain'] ) / (0.01*len(test_set.index)) )))
test_set['predicted_pruned_tree_variance'] = test_set.apply(tree_accuracy, axis=1, args=(pruned_tree_variance,'1') ) 
print( 'Accuracy with pruned Variance gain tree ' + (str( sum(test_set['Class']==test_set['predicted_pruned_tree_variance'] ) / (0.01*len(test_set.index)) )))


XO = 0 : 
| XM = 0 : 
| | XF = 0 : 
| | | XB = 0 : 
| | | | XG = 0 : 1 
| | | | XG = 1 : 
| | | | | XD = 0 : 
| | | | | | XS = 0 : 1 
| | | | | | XS = 1 : 
| | | | | | | XC = 0 : 0 
| | | | | | | XC = 1 : 
| | | | | | | | XH = 0 : 1 
| | | | | | | | XH = 1 : 0 
| | | | | XD = 1 : 
| | | | | | XP = 0 : 
| | | | | | | XE = 0 : 1 
| | | | | | | XE = 1 : 0 
| | | | | | XP = 1 : 1 
| | | XB = 1 : 
| | | | XD = 0 : 1 
| | | | XD = 1 : 
| | | | | XI = 0 : 1 
| | | | | XI = 1 : 
| | | | | | XG = 0 : 0 
| | | | | | XG = 1 : 1 
| | XF = 1 : 1 
| XM = 1 : 
| | XB = 0 : 
| | | XD = 0 : 
| | | | XG = 0 : 
| | | | | XF = 0 : 1 
| | | | | XF = 1 : 
| | | | | | XJ = 0 : 
| | | | | | | XN = 0 : 0 
| | | | | | | XN = 1 : 
| | | | | | | | XE = 0 : 
| | | | | | | | | XK = 0 : 1 
| | | | | | | | | XK = 1 : 0 
| | | | | | | | XE = 1 : 1 
| | | | | | XJ = 1 : 
| | | | | | | XC = 0 : 
| | | | | | | | XT = 0 : 
| | | | | | | | | XL = 0 : 
| | | | | | | | | | XE = 0 : 
| | | | | | | | | | | XI = 0 : 1 
| | | |